<a href="https://colab.research.google.com/github/YoungHyunKoo/GEE_remote_sensing/blob/main/Week4/4_2_Object_based_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **[GEO 6083] Remote Sensing Imge Processing - Spring 2024**
# **WEEK 4-2. Object-based image classification**

### OBJECTIVES
1. Implement image segmentation
2. Implement object-based segmentation

Credited by Younghyun Koo (kooala317@gmail.com)

## **Object-based classification**
Traditional pixel-based image classification assigns a land cover class per pixel. All pixels are the same size, same shape, and don’t consider their neighbors during the training process. However, **object-based** image classification can be useful approach to obtain more realistic classification results by grouping several pixels as *objectives*. This approach is closer to what your eyes really perceive.

In [ ]:
# Import ee library
import ee

# Authenticate
ee.Authenticate()

# Initialize with your own project.
ee.Initialize(project = "utsa-spring2024")

In [ ]:
# Import geemap library
import geemap

In [ ]:
# Import geopandas and pandas library
import geopandas as gpd
import pandas as pd

In [ ]:
# Mount to Google Drive (save files)
from google.colab import drive
drive.mount('/content/drive')

## **Implement Image segmentation**

In this example, we will use the NAIP: National Agriculture Imagery Program data. [NAIP: National Agriculture Imagery Program](https://developers.google.com/earth-engine/datasets/catalog/USDA_NAIP_DOQQ) This data has a very fine spatial resolution (around 1 m) and consists of four bands (RGB + NIR).


In [ ]:
AOI = ee.Geometry.Polygon(
    [
        [-91.1028433068419, 40.810549087005384],
        [-91.0761499627257, 40.810549087005384],
        [-91.0761499627257, 40.82444972968615],
        [-91.1028433068419, 40.82444972968615],
        [-91.1028433068419, 40.810549087005384]
     ]
)

# import image data
dataset = ee.ImageCollection('USDA/NAIP/DOQQ')\
.filterDate('2014-01-01', '2014-12-31')\
.filterBounds(AOI)

img = dataset.mean().clip(AOI)

trueColor = dataset.select(['R', 'G', 'B']);
trueColorVis = {
  'min': 0.0,
  'max': 255.0,
};

Map = geemap.Map()

Map.centerObject(dataset, 15);
Map.addLayer(img, trueColorVis, 'True Color')

Map

We will add an NDVI band as an additional band used for the classification.

In [ ]:
# Add NDVI band
ndvi = img.normalizedDifference(["N", "R"]).rename('NDVI')

Map.addLayer(ndvi, {'min':-0.5, 'max':0.5, 'palette': ['red', 'white', 'green']}, 'NDVI')

Map

You will add one more band: entropy, which represents the texture. [Image entropy](https://developers.google.com/earth-engine/apidocs/ee-image-entropy)

Entropy is computed as $-Σ(p * log2(p))$, where $p$ is the normalized probability of occurrence of the values encountered in each window.

In [ ]:
# Add entropy band

square = ee.Kernel.square(radius = 4);

entropy = img.select('N').toByte().entropy(square);
Map.addLayer(entropy,
             {'min': 1, 'max': 5, 'palette': ['red', 'blue']},
             'entropy');

Map

In [ ]:
bands = ['R', 'G', 'B', 'N']
FullImage = img.select(bands).float().divide(255)
FullImage = FullImage.addBands(entropy) #.rename('entropy')
FullImage = FullImage.addBands(ndvi) #.rename('ndvi')

Now we will conduct image segmentation. GEE has provided an image segmentation method called SNIC (Simple Non-Iterative Clustering). You can find more details about this algorithm in the following links:
- ["Superpixels and Polygons Using Simple Non-iterative Clustering
Publisher" by Radhakrishna Achanta](https://ieeexplore.ieee.org/document/8100003)
- [ee.Algorithms.Image.Segmentation.SNIC](https://developers.google.com/earth-engine/apidocs/ee-algorithms-image-segmentation-snic)

In [ ]:
# Initial superpixel: Deterine the size of segments
seeds = ee.Algorithms.Image.Segmentation.seedGrid(40);

# Run SNIC image segmentation
snic = ee.Algorithms.Image.Segmentation.SNIC(
    image = FullImage,
    compactness = 1,
    connectivity = 4,
    seeds = seeds
)

clusters_snic = snic.select("clusters")

# Convert SNIC result into vector polygons
vectors = clusters_snic.reduceToVectors(
    geometryType = 'polygon',
    reducer = ee.Reducer.countEvery(),
    scale = 1,
    maxPixels = 1e13,
    geometry = AOI,
)

# Draw outline features
empty = ee.Image().byte()
outline = empty.paint(
    featureCollection = vectors,
    color = 1,
    width = 1
)

Map.addLayer(outline, {'palette': 'black'}, 'segments')

Map

In [ ]:
# Convert SNIC result into vector polygons
vectors = clusters_snic.reduceToVectors(
    geometryType = 'polygon',
    reducer = ee.Reducer.countEvery(),
    scale = 1,
    maxPixels = 1e13,
    geometry = AOI,
)

Now the segmentation is done! We will label these segments and use them as training datasets for a supervised classifier.

## **Get training samples**

As we did in the previuos tutorial, we will manually digitize the training samples. Please keep in mind that we will label *segments* in this tutorial, whereas we labeled *pixels* in the previous tutorial. Here, we will define 5 classes: (1) water, (2) urban, (3) grass, (4) tree, and (5) agriculture.

(1) Water

Please put point pins to any water segments.

In [ ]:
Map = geemap.Map()

Map.centerObject(img, 15)

Map.addLayer(img, trueColorVis, "True color")
Map.addLayer(outline, {'palette': 'black'}, 'segments')

Map

In [ ]:
water = ee.FeatureCollection(Map.draw_features)
print(water.size().getInfo())

(2) Urban

Next, let's find some urban segments. Urban class includes roads and buildings.

In [ ]:
Map = geemap.Map()

Map.centerObject(img, 15)
Map.addLayer(img, trueColorVis, "True color")
Map.addLayer(outline, {'palette': 'black'}, 'segments')

Map

In [ ]:
urban = ee.FeatureCollection(Map.draw_features)
print(urban.size().getInfo())

(3) Grass

Next, let's find out some grass segments. Please note that you should exclude the agriculture area on the east side because we will define this agriculture site as an independent class.

In [ ]:
Map = geemap.Map()

Map.centerObject(img, 15)
Map.addLayer(img, trueColorVis, "True color")
Map.addLayer(outline, {'palette': 'black'}, 'segments')

Map

In [ ]:
grass = ee.FeatureCollection(Map.draw_features)
print(grass.size().getInfo())

(4) Tree

In [ ]:
Map = geemap.Map()

Map.centerObject(img, 15)
Map.addLayer(img, trueColorVis, "True color")
Map.addLayer(outline, {'palette': 'black'}, 'segments')

Map

In [ ]:
tree = ee.FeatureCollection(Map.draw_features)
print(tree.size().getInfo())

(5) Agriculture

In [ ]:
Map = geemap.Map()

Map.centerObject(img, 15)
Map.addLayer(img, trueColorVis, "True color")
Map.addLayer(outline, {'palette': 'black'}, 'segments')

Map

In [ ]:
agriculture = ee.FeatureCollection(Map.draw_features)
print(agriculture.size().getInfo())

In [ ]:
# Define the class number to each class
def add_water(feature):
  return feature.set("Class", 0)

def add_urban(feature):
  return feature.set("Class", 1)

def add_grass(feature):
  return feature.set("Class", 2)

def add_tree(feature):
  return feature.set("Class", 3)

def add_agriculture(feature):
  return feature.set("Class", 4)

water = water.map(add_water)
urban = urban.map(add_urban)
grass = grass.map(add_grass)
tree = tree.map(add_tree)
agriculture = agriculture.map(add_agriculture)

In [ ]:
# Combine all points into a single feature collection
train_points = water.merge(urban).merge(grass).merge(tree).merge(agriculture)

In [ ]:
# Convert the feature collection into geodataframe
gdf_train = geemap.ee_to_gdf(train_points)
gdf_train["latitude"] = gdf_train.geometry.y
gdf_train["longitude"] = gdf_train.geometry.x
gdf_train

In [ ]:
# Save table as csv file
gdf_train.to_csv('drive/MyDrive/samples_object.csv')

# Restore samples from csv file
df = pd.read_csv('drive/MyDrive/samples_object.csv', index_col = 0)

# Convert pandas dataframe to geopandas geodataframe (contains geometry information)
s = gpd.GeoSeries.from_wkt(df.geometry, crs=4326) # CRS: WGS84 (latitude, longitude)
gdf_train = gpd.GeoDataFrame(data=df, geometry=s)

gdf_train

## **Train classifier in object-level**

Now we have the training dataset as a geodataframe. However, since the digitized training samples have point geometries, we need to find the segment corresonding to each digitized point. We can do this by checking the spatial relationships between geopandas dataframe.

In [ ]:
# Convert the segments into geodataframe
segments = geemap.ee_to_gdf(vectors)

In [ ]:
# For all segments, we will find what segments contain the training points
for i in range(0, len(segments)):
  segment = segments.loc[i, :].geometry

  # Compare with all training sample points
  for j in range(len(gdf_train)):
    point = gdf_train.loc[j, :].geometry

    if point.within(segment):
      segments.loc[i, "belongs"] = "TRUE"
      segments.loc[i, "Class"] = gdf_train.loc[j, "Class"]

      # break

In [ ]:
# Geodataframe of segments that have the intersecting training points.
gdf = segments[segments["belongs"] == "TRUE"]
gdf

In [ ]:
len(gdf)

In [ ]:
# Convert the polygon geodataframe of selected segments into earth engine feature collection
gdf = gdf.set_crs('EPSG:4326')
train_poly = geemap.geopandas_to_ee(gdf)

In [ ]:
train_areas = train_poly.reduceToImage(
    properties = ['Class'],
    reducer = ee.Reducer.first()
).rename('Class').toInt()

In [ ]:
# Convert segmented vector into image
predict_image = vectors.reduceToImage(
    properties = ['label'],
    reducer = ee.Reducer.first()
).rename('id').toInt();

FullImage = FullImage.addBands(predict_image)

# Calculate mean of the segment
FullImage_mean = FullImage.reduceConnectedComponents(
    reducer = ee.Reducer.mean(),
    labelBand = 'id'
);

# Calculate std of the segment
FullImage_std = FullImage.reduceConnectedComponents(
    reducer = ee.Reducer.stdDev(),
    labelBand = 'id'
);

# Calculate median of the segment
FullImage_median = FullImage.reduceConnectedComponents(
    reducer = ee.Reducer.median(),
    labelBand = 'id'
);

# Input image - combination of mean, std, and median of full image
Pred_bands = ee.Image.cat([
  FullImage_mean,
  FullImage_std,
  FullImage_median
]).float()

In [ ]:
FullImage.getInfo()

## **Apply classifier to the segmented polygons**

We will use Random Forest classifier for this example. [ee.Classifier.smileRandomForest](https://developers.google.com/earth-engine/apidocs/ee-classifier-smilerandomforest)

In [ ]:
# Training images
clip_Image = Pred_bands.clip(train_poly)

train_areas = train_areas.addBands(clip_Image)

predictionBands = Pred_bands.bandNames();

classifierTraining = train_areas.select(predictionBands).sampleRegions(collection = train_poly, properties = ['Class'], scale = 2);

# Train random forest classifier
RF = ee.Classifier.smileRandomForest(50).train(features = classifierTraining, classProperty = 'Class', inputProperties = predictionBands);

# Apply random forest classifier
classified_RF = Pred_bands.select(predictionBands).classify(RF);

print("Classification has been done!!")



In [ ]:
Map = geemap.Map()

Map.centerObject(dataset, 15);

Map.addLayer(img, trueColorVis, 'True Color')

vis_RF = {'min': 0, 'max': 4, 'palette': [ 'blue', 'red', 'orange', 'green', 'yellow']}

Map.addLayer(classified_RF, vis_RF, "OBIA_RF");
Map

## References
- https://joaootavionf007.medium.com/object-based-image-analysis-on-google-earth-engine-1b80e9cb7312